In [99]:
import numpy as np
import pandas as pd
import requests
import datetime
import re
import time
from fake_useragent import UserAgent
import random
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [175]:
today = datetime.date.today()

try:
    with open('QuotaData_Setting.txt',mode='r', encoding='utf-8') as f:
        quota_setting = f.readlines()
except:
    print('未定義QuotaData_Setting    由系統建立預設值設定檔')
    last_month_date = (today - pd.tseries.offsets.MonthEnd(1)).replace(day=1).date()
    with open("QuotaData_Setting.txt", "a") as f:
        f.write(f"START:{last_month_date.strftime('%Y-%m-%d')}\nCREDIT_TABLE:credit_data_table.pickle\nSYSTEM_TABLE:system_table.pickle\nFetch_CSV:")
    with open('QuotaData_Setting.txt',mode='r') as f:
        quota_setting = f.readlines()
    quota_table = pd.DataFrame({},columns=["日期",'股票代號', 
                                           '股票名稱', '前日餘額', '賣出', '買進', '現券',
                                           '今日餘額', '限額', '前日餘額', '當日賣出',
                                           '當日還券', '當日調整', '當日餘額', 
                                           '次一營業日可限額', '備註'])
    system_table = pd.DataFrame({},columns=["日期", '證券代號',
                                             '證券名稱', '前日借券餘額股', '本日異動股借券',
                                             '本日異動股還券', '本日借券餘額股', 
                                             '本日收盤價單位', '借券餘額市值單位', '市場別'])
    quota_table.to_pickle('credit_data_table.pickle')
    system_table.to_pickle('system_table.pickle')


In [176]:
def set_header_user_agent():
    user_agent = UserAgent()
    return user_agent.random

In [177]:
def get_quote(date):
    parse_date = date.strftime(format='%Y%m%d')
    date = date.strftime(format='%Y-%m-%d')
    user_agent = set_header_user_agent()
    res = requests.get(f'https://www.twse.com.tw/exchangeReport/TWT93U?response=csv&date={parse_date}',
                      headers={ 'user-agent': user_agent})
    try:   
        a = res.text.split('\r\n')[1:-6] #原始文字清洗，清除首行及表尾說明語句
        b = [a.replace('=', "") for a in a]  #非個股其證券代號名稱前會有等號，予以刪除
        c = [c.split('","') for c in b] #每一列分割欄位
        df = pd.DataFrame(c[2:], columns=c[1])
        df = df.dropna()
        df.columns = [i.strip('",') for i in df.columns]
        df = df.applymap(lambda x:x.strip('",').replace(',',""))
        df = df[df.股票名稱!='合計']
        df.insert(0,'日期', date)
        return df
    except:
        return None


In [178]:
def get_system(date):
    parse_date = date.strftime(format='%Y%m%d')
    date = date.strftime(format='%Y-%m-%d')
    user_agent = set_header_user_agent()
    res = requests.get(f'https://www.twse.com.tw/exchangeReport/TWT72U?response=csv&date={date}&selectType=SLBNLB',
                      headers={ 'user-agent': user_agent})
    try:
        a = res.text.split('\r\n')[1:-6] #原始文字清洗，清除首行及表尾說明語句
        b = [a.replace('=', "") for a in a]  #非個股其證券代號名稱前會有等號，予以刪除
        c = [c.split('","') for c in b] #每一列分割欄位
        df = pd.DataFrame(c[1:], columns=c[0])
        df.columns = [i.strip('",') for i in df.columns]
        df = df.applymap(lambda x:x.strip('",').replace(',',""))
        df = df[df.證券代號!='合計'] 
        df.insert(0,'日期', date)
        return df
    except:
        pass

開始寫表格

In [179]:
quota_setting = [i.strip('\n').split(':')[1] for i in quota_setting]

In [180]:
quota_table = pd.read_pickle(quota_setting[1])
system_table = pd.read_pickle(quota_setting[2])

In [191]:
initial = False
if (len(quota_table)==0) & (len(quota_table)==0):
    initial = True

In [213]:
if initial ==True:
    print(f'-------------初次下載  由 {start_date.strftime(format="%Y-%m-%d")} 開始下載--------------')
    parse_date = start_date
    while(parse_date<=today):
        print(f'-----------{parse_date.strftime(format="%Y-%m-%d")}------------')
        temp = get_quote(parse_date)
        if type(temp) == pd.core.frame.DataFrame:
            quota_table = quota_table.append(temp)
            print(f'---成功下載 : quota_table----')
        else:
            print(f'---無法下載 : quota_table----')
            
        time.sleep(random.randint(2, 5))
        
        temp = get_system(parse_date)
        if type(temp) == pd.core.frame.DataFrame:
            system_table = system_table.append(temp)
            print(f'---成功下載 : system_table----')
        else:
            print(f'---無法下載 : system_table----')
        
        time.sleep(random.randint(3, 6))
        
        parse_date= parse_date+datetime.timedelta(1)
    print('-----------儲存資料----------')

else:
    quota_lastest_date = pd.to_datetime(quota_table.日期).max().date()
    system_lastest_date = pd.to_datetime(system_table.日期).max().date
    
    parse_date = min(parse_date, system_lastest_date)
    print(f'-------------接續前次紀錄下載  由 {parse_date.strftime(format="%Y-%m-%d")} 開始下載--------------')
    
    while(parse_date<=today):
        print(f'-----------{parse_date.strftime(format="%Y-%m-%d")}------------')
        temp = get_quote(parse_date)
        if type(temp) == pd.core.frame.DataFrame:
            quota_table = quota_table.append(temp)
            print(f'---成功下載 : quota_table----')
        else:
            print(f'---無法下載 : quota_table----')
            
        time.sleep(random.randint(2, 5))
        
        temp = get_system(parse_date)
        if type(temp) == pd.core.frame.DataFrame:
            system_table = system_table.append(temp)
            print(f'---成功下載 : system_table----')
        else:
            print(f'---無法下載 : system_table----')
        
        time.sleep(random.randint(3, 6))
        parse_date= parse_date+datetime.timedelta(1)
    
    quota_table.to_pickle('credit_data_table.pickle')
    system_table.to_pickle('system_table.pickle')
    print('-----------儲存資料----------')
    

-------------初次下載  由 2020-11-01 開始下載--------------
-----------2020-11-01------------


ConnectionError: ('Connection aborted.', ConnectionResetError(10054, '遠端主機已強制關閉一個現存的連線。', None, 10054, None))